In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_info)

In [ ]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

In [ ]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [ ]:
!pip install tensorflow-addons

In [ ]:
from tensorflow import device
from tensorflow_addons import layers as new_layers
from tensorflow.keras import losses
from tensorflow.keras import models

import csv
import gc
import h5py
import imageio
import numpy as np
import os

In [ ]:
town = 'Berlin'  #@param ['Berlin', 'Istanbul', 'Moscow']

model_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/checkpoints/UNet12_2/model_9.h5'.format(town)
with device('gpu:0'):
    model = models.load_model(model_path)
loss = losses.MeanSquaredError()

files = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/files/validation'.format(town)

logs = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/logs/UNet12_2/validation/logs.csv'.format(town)

movies = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/movies/validation/UNet12_2'.format(town)

movie_names = ['ne_volume', 'ne_average_speed', 'se_volume', 'se_average_speed', 
               'sv_volume', 'sv_average_speed', 'nv_volume', 'nv_average_speed']

In [ ]:
def get_file_names(files):
    return os.listdir(files)

In [ ]:
def get_data(file_path):
    file = h5py.File(file_path, 'r')
    group_key = list(file.keys())[0]
    data = np.array(file[group_key][:], dtype=np.float32)  # (288, 495, 436, 9)
    file.close()
    return data

In [ ]:
def get_validation_data(data):
    data = np.array(np.split(data, 12))  # split in 12 batches of 12 + 12 timestamps
    inputs = data[:, :12, :, :, :]
    outputs = data[:, 12:, :, :, :]
    outputs = np.take(outputs, np.arange(8), axis=-1)  # keep only the dynamic channels
    inputs = np.moveaxis(inputs, 1, -1).reshape((12, 495, 436, -1))  # combine the timestamps with the channels
    outputs = np.moveaxis(outputs, 1, -1).reshape((12, 495, 436, -1))  # combine the timestamps with the channels
    return inputs / 255.0, outputs / 255.0

In [ ]:
def get_restored_validation_data(data):
    data *= 255.0
    data = np.moveaxis(data.reshape((12, 495, 436, 8, 12)), -1, 1)  # (12, 12, 495, 436, 8)
    return data.reshape((-1, 495, 436, 8))  # (144, 495, 436, 8)

In [ ]:
def get_mask(data):
    data = np.array(np.split(data, 12))  # split in 12 batches of 12 + 12 timestamps
    data = data[:, :12, :, :, :]  # keep only the inputs
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)  # take only the average speed channels
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)  # (144 * 4, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [ ]:
def create_movie(actual, prediction, mask, movies, file_path):
    actual = np.sum(actual, axis=-1, dtype=np.uint8)
    prediction = np.sum(prediction, axis=-1, dtype=np.uint8)
    prediction *= mask
    frames = np.concatenate((actual, prediction), axis=-1)
    path = os.path.join(movies, os.path.basename(file_path)[:-7] + '.gif')
    imageio.mimsave(path, [frame for frame in frames], fps=3)
    print('created movie:', path)

In [ ]:
def create_movies(actual, prediction, mask, movies, file_path):
    for channel in range(8):
        first = np.squeeze(actual[:, :, :, channel])
        second = np.squeeze(prediction[:, :, :, channel])
        second *= mask
        frames = np.concatenate((first, second), axis=-1)
        frames = np.clip(frames * 3.0, 0.0, 255.0).astype('uint8')
        path = os.path.join(movies, os.path.basename(file_path)[:-6] + movie_names[channel] + '.gif')
        imageio.mimsave(path, [frame for frame in frames], fps=3)
        print('created movie:', path)

In [ ]:
def get_validation_loss(model, loss, file_path, movies):
    data = get_data(file_path)
    inputs, actual = get_validation_data(data)
    with device('gpu:0'):
        prediction = model.predict(inputs)
        prediction = np.clip(prediction, 0.0, 1.0)
    gc.collect()
    actual = np.take(actual, [0, 1, 2, 5, 8, 11], axis=0)
    prediction = np.take(prediction, [0, 1, 2, 5, 8, 11], axis=0)
    return loss(actual, prediction).numpy()

In [ ]:
file_names = get_file_names(files)

log_file = open(logs, 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['file', 'loss'])
log_file.flush()

losses = np.zeros(shape=(len(file_names),), dtype=np.float64)

for index, file_name in enumerate(file_names):
    print('file:', index)
    result = get_validation_loss(model, loss, os.path.join(files, file_name), movies)
    print('validation loss:', result)
    log_writer.writerow([file_name, result])
    log_file.flush()
    losses[index] = result

mean = np.mean(losses)
median = np.median(losses)
print('validation loss mean:', mean)
print('validation loss median:', median)

log_writer.writerow(['mean', mean])
log_writer.writerow(['median', median])
log_file.flush()

log_file.close()

In [ ]:
file_path = os.path.join(files, file_names[np.argmin(losses)])
data = get_data(file_path)
mask = get_mask(data)
inputs, actual = get_validation_data(data)
with device('gpu:0'):
    prediction = model.predict(inputs)
    prediction = np.clip(prediction, 0.0, 1.0)
gc.collect()
actual = get_restored_validation_data(actual)
prediction = get_restored_validation_data(prediction)

In [ ]:
create_movie(actual, prediction, mask, movies, file_path)

In [ ]:
create_movies(actual, prediction, mask, movies, file_path)